# 如何组合提示词

:::info 预备知识

本指南假设您已熟悉以下概念：

- [提示词模板](/docs/concepts/prompt_templates)

:::

LangChain 提供了一个用户友好的界面，用于将提示词的不同部分组合在一起。您可以使用字符串提示词或聊天提示词进行组合。以这种方式构建提示词可以方便组件的重复使用。

## 字符串提示组合

在使用字符串提示时，每个模板会被连接在一起。你可以直接操作提示或字符串（列表中的第一个元素必须是提示）。

In [3]:
import { PromptTemplate } from "@langchain/core/prompts";

const prompt = PromptTemplate.fromTemplate(`Tell me a joke about {topic}, make it funny and in {language}`)

prompt

PromptTemplate {
  lc_serializable: true,
  lc_kwargs: {
    inputVariables: [ "topic", "language" ],
    templateFormat: "f-string",
    template: "Tell me a joke about {topic}, make it funny and in {language}"
  },
  lc_runnable: true,
  name: undefined,
  lc_namespace: [ "langchain_core", "prompts", "prompt" ],
  inputVariables: [ "topic", "language" ],
  outputParser: undefined,
  partialVariables: undefined,
  templateFormat: "f-string",
  template: "Tell me a joke about {topic}, make it funny and in {language}",
  validateTemplate: true
}

In [4]:
await prompt.format({ topic: "sports", language: "spanish" })

"Tell me a joke about sports, make it funny and in spanish"

## 聊天提示词构成

一个聊天提示由一系列消息组成。与上面的例子类似，我们可以串联聊天提示模板。每个新元素都是最终提示中的新消息。

首先，让我们用一个 [`SystemMessage`](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html) 初始化 [`ChatPromptTemplate`](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)。

In [5]:
import { AIMessage, HumanMessage, SystemMessage} from "@langchain/core/messages"

const prompt = new SystemMessage("You are a nice pirate")

然后你可以轻松地将其与其他消息*或*消息模板组合创建一个管道。
当没有变量需要格式化时使用`BaseMessage`，当有变量需要格式化时使用`MessageTemplate`。你也可以直接使用一个字符串（注意：这将自动被推断为一个[`HumanMessagePromptTemplate`](https://api.js.langchain.com/classes/langchain_core.prompts.HumanMessagePromptTemplate.html)）。

In [6]:
import { HumanMessagePromptTemplate } from "@langchain/core/prompts"

const newPrompt = HumanMessagePromptTemplate.fromTemplate([prompt, new HumanMessage("Hi"), new AIMessage("what?"), "{input}"])

在底层，这会创建一个ChatPromptTemplate类的实例，因此你可以像以前一样使用它！

In [8]:
await newPrompt.formatMessages({ input: "i said hi" })

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: [
        { type: "text", text: "You are a nice pirate" },
        { type: "text", text: "Hi" },
        { type: "text", text: "what?" },
        { type: "text", text: "i said hi" }
      ],
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: [
      { type: "text", text: "You are a nice pirate" },
      { type: "text", text: "Hi" },
      { type: "text", text: "what?" },
      { type: "text", text: "i said hi" }
    ],
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]

## 使用 PipelinePrompt

LangChain 包含一个名为 [`PipelinePromptTemplate`](https://api.js.langchain.com/classes/_langchain_core.prompts.PipelinePromptTemplate.html) 的类，当你想要重复使用提示的一部分时，它会非常有用。一个 PipelinePrompt 主要由两部分组成：

- 最终提示：返回的最终提示
- 管线提示：由字符串名称和提示模板组成的元组列表。每个提示模板将被格式化，然后以相同名称的变量形式传递给后续的提示模板。

In [1]:
import {
    PromptTemplate,
    PipelinePromptTemplate,
  } from "@langchain/core/prompts";
  
const fullPrompt = PromptTemplate.fromTemplate(`{introduction}

{example}

{start}`);

const introductionPrompt = PromptTemplate.fromTemplate(
`You are impersonating {person}.`
);

const examplePrompt =
PromptTemplate.fromTemplate(`Here's an example of an interaction:
Q: {example_q}
A: {example_a}`);

const startPrompt = PromptTemplate.fromTemplate(`Now, do this for real!
Q: {input}
A:`);

const composedPrompt = new PipelinePromptTemplate({
pipelinePrompts: [
    {
    name: "introduction",
    prompt: introductionPrompt,
    },
    {
    name: "example",
    prompt: examplePrompt,
    },
    {
    name: "start",
    prompt: startPrompt,
    },
],
finalPrompt: fullPrompt,
});
  

  

In [2]:
const formattedPrompt = await composedPrompt.format({
    person: "Elon Musk",
    example_q: `What's your favorite car?`,
    example_a: "Telsa",
    input: `What's your favorite social media site?`,
  });
  
  
console.log(formattedPrompt);


You are impersonating Elon Musk.

Here's an example of an interaction:
Q: What's your favorite car?
A: Telsa

Now, do this for real!
Q: What's your favorite social media site?
A:


## 下一步

你现在已经学会了如何将提示词组合在一起。

接下来，请查看本节中有关提示模板的其他操作指南，例如[将少量示例添加到提示模板](/docs/how_to/few_shot_examples_chat)。